In [1]:
import numpy as np
import matplotlib.pyplot as plt 

%pylab inline
plt.style.use('seaborn-dark')
plt.rc('font', size=15)


import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
def PrintTriangular(mas, i, lines=None):
    if lines is None:
        lines = len(mas)
        
    for line in range(lines):
        for n in range(line + 1 - i):
            print('{0:7.4f}'.format(mas[line][n]), end=' ')
        print()

In [3]:
def Integration(u, a, b, N):
    h = (b - a) / N
    x = linspace(a, b, N+1)
    integral = 0.
    for n in range(1, N+1):
        integral = integral + (u(x[n-1]) + u(x[n])) / 2*h 
    return integral

In [4]:
def RicherdsonExtrapolation(func, a, b, EPS=10, p=2, q=2, r=2, N=2, S=15):
    U = np.zeros((S,S))
    R = np.zeros((S,S))
    p_eff = np.zeros((S,S))

    s = 1
    U[0][0] = Integration(func, a, b, N)
    while True:
        U[s][0] = Integration(func, a, b, r**s*N)

        for n in range(s):
            R[s][n] = (U[s][n] - U[s-1][n]) / (r**(p + n*q) - 1)
            
            p_eff[s][n] = log(abs(R[s-1][n]/R[s][n])) / log(r)
            
            U[s][n + 1] = U[s][n] + R[s][n]
            
            if abs(R[s][n]) < (0.1)**EPS:
                return (U[s][n], R[s][n], p_eff[s][n], p*(s-1)+n*q | p, s, n, U, R, p_eff)

            
        
        s += 1
        if s > 13: 
            print('oops')
            return (U[s-1][s-1], R[s-1][s-1], p_eff[s-1][s-3], p + s*q, s, s, U, R, p_eff)



In [5]:
f = lambda x: 3*np.sin(x)*x**2*np.exp(x)/x

ans = RicherdsonExtrapolation(f, a=0.1, b=1., EPS=18)
re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
Эффективный порядок точности: {2:7.10f}\n \
Теоретический порядок точности: {3:7.5f}\n \
".format(ans[0], ans[1], ans[2], ans[3])
print(re)
PrintTriangular(ans[-3], 0, ans[4])
PrintTriangular(ans[-2], 1, ans[4])
PrintTriangular(ans[-1], 2, ans[4])

 Ответ: 1.9299559322
 Погрешность: -0.0000000000
 Эффективный порядок точности: 3.2094533656
 Теоретический порядок точности: 18.00000
 
 2.2241 
 2.0035  1.9300 
 1.9483  1.9300  1.9300 
 1.9346  1.9300  1.9300  1.9300 
 1.9311  1.9300  1.9300  1.9300  1.9300 
 1.9302  1.9300  1.9300  1.9300  1.9300  1.9300 

-0.0735 
-0.0184 -0.0000 
-0.0046 -0.0000  0.0000 
-0.0011 -0.0000  0.0000  0.0000 
-0.0003 -0.0000  0.0000  0.0000 -0.0000 


 1.9994 
 1.9999  4.6638 
 2.0000  4.2282  5.9980 
 2.0000  4.0632  5.9996  8.2015 
